In [2]:
import cv2
import numpy as np
import time
import imutils
from joblib import load
from dataset_io import *
import timeit

In [3]:
def pyramid(image, scale=1.5, minSize=(30, 30)):
    yield image
    while True:
        w = int(image.shape[1] / scale)
        image = imutils.resize(image, width=w)
        if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]:
            break
        yield image
        
def sliding_window(image, stepSize, windowSize):
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

In [10]:
(winW, winH) = (120,24)

clf = load('rand_forest_clf.joblib') 
clf.set_params(n_jobs=-1)

def predict(filename, scene_name, label_resolution, rects):
    image=imread_resized(scene_name, filename, label_resolution)
    channels=read_channels(scene_name, filename)
    for resized in pyramid(image, scale=1.5):
        X=[]
        for (x, y, window) in sliding_window(resized, stepSize=32, windowSize=(winW, winH)):
            # if the window does not meet our desired window size, ignore it
            if window.shape[0] != winH or window.shape[1] != winW:
                continue
            channels_window=channels[y:y + winH, x:x + winW]
            X.append(channels_window.flatten())
            clone = resized.copy()
#             cv2.rectangle(clone, (x, y), (x + winW, y + winH), (255, 255, 255), 2)
#             cv2.imshow("Window", clone)
#             cv2.waitKey(1)
        if X:
            y_pred=clf.predict(X) #TODO may be per image, will be faster
    print("filename")

In [11]:
def predict_single():
    row=walk_dataset()[0]
    predict(*row)
    
timeit.timeit(predict_single, number=10)
# features only - 2.0s
# classify per image - 3.36s
# classify per window - 9.920s

filename
filename
filename
filename
filename
filename
filename
filename
filename
filename


9.198062949000814

In [ ]:
[predict(*row) for row in walk_dataset()]